In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import json
import pandas as pd
from enbios.base.experiment import Experiment
import bw2data

from enbios.bw2.util import report
from enbios.models.experiment_models import ExperimentData

In [ ]:
# get an overview of brighway projects and databases
report()

In [ ]:
# select the brightway project and database (some ecoinvent database)
PROJECT_NAME = "ecoinvent_391"
DATABASE = "ecoinvent_391_cutoff"

bw2data.projects.set_current(PROJECT_NAME)
db = bw2data.Database(DATABASE)

In [ ]:
wind_turbines_spain = db.search(
    "electricity production, wind, 1-3MW turbine, onshore", filter={"location": "ES"}
)[:1]
wind_turbines_spain

In [ ]:
solar_spain = db.search("solar", filter={"location": "ES"})[:2]
solar_spain

In [ ]:
# for the experiment we need to create a list of activities (or a dict, where the keys represent the aliases)
# We need to add the codes, otherwise the brightway search will not be not uniquely identify the activities
# adding name is just for convenience
experiment_activities = []

for activity in wind_turbines_spain + solar_spain:
    experiment_activities.append(
        {
            "id": {
                "name": activity["name"],
                "code": activity["code"],
                "alias": activity["name"],
            }
        }
    )

experiment_activities

In [ ]:
# select 2 random methods and convert them into the form for enbios2
methods = [bw2data.methods.random() for _ in range(1)]
experiment_methods = [{"id": method} for method in methods]

experiment_methods

In [ ]:
hierarchy = {
    "wind": [wind_act["name"] for wind_act in wind_turbines_spain],
    "solar": [solar_act["name"] for solar_act in solar_spain],
}

In [ ]:
from random import randint


def create_random_scenario():
    return {
        "activities": {
            act["id"]["alias"]: ["kilowatt_hour", randint(1, 10)]
            for act in experiment_activities
        }
    }


scenarios = [create_random_scenario() for _ in range(8)]

In [ ]:
# let's store the raw data, because we want to modify it later
raw_data = {
    "bw_project": PROJECT_NAME,
    "activities": experiment_activities,
    "methods": experiment_methods,
    "hierarchy": hierarchy,
    "scenarios": scenarios,
}

raw_data

In [ ]:
# create a experiment object. This will validate the activities, their outputs, the methods and the scenarios.
exp: Experiment = Experiment(raw_data)

In [ ]:
print(exp.info())

In [ ]:
exp = Experiment(raw_data)

In [ ]:
_ = exp.run()

In [ ]:
exp.execution_time

In [ ]:
from pathlib import Path

exp.results_to_csv("s1.csv", level_names=["root", "technology", "activity"])
df = pd.read_csv("s1.csv").fillna("")
Path("s1.csv").unlink()
df

In [ ]:
from enbios.generic.files import DataPath
import pickle

pickle.dump(exp, DataPath("test_data/exp.pkl").open("wb"))

In [ ]:
activities = list(exp.scenarios[0].result_tree.get_leaves())
[a._data for a in activities]

In [ ]:
exp.scenarios[0].rearrange_results(
    {
        "group1": [
            "electricity production, wind, >3MW turbine, onshore",
            "electricity production, wind, 1-3MW turbine, onshore",
            "electricity production, wind, 1-3MW turbine, offshore",
        ],
        "group2": [
            "electricity production, wind, <1MW turbine, onshore",
            "electricity production, solar tower power plant, 20 MW",
            "electricity production, solar thermal parabolic trough, 50 MW",
        ],
    }
)